In [62]:
import requests 
import pandas as pd
import numpy as np
import math
from pprint import pprint
import re
import os 
from statistics import mean, median, mode
%matplotlib inline
import matplotlib.pyplot as plt
import json

In [63]:
MAX_RECORDS = 200
latmin = 43.6406
latmax = 43.736439
lngmax = -79.252357       
lngmin = -79.549301
records = 2500
records_per_page = min(records, MAX_RECORDS)
pages = math.ceil(records / MAX_RECORDS)

maindf = pd.DataFrame({"Address":''},index=[np.arange(0, records)])
maindf["Sale Price"]= ""    
maindf['Mls Number']=""
maindf['Near By Ammenities']= ""
maindf["Ownership Type"] = ""
maindf["Parking"] = ""
maindf["Bathrooms"] = ""
maindf["Bedrooms"] = ""

for page in np.arange(pages):
    url = 'https://api2.realtor.ca/Listing.svc/PropertySearch_Post'
    body = {
        "CultureId":1,
        "ApplicationId":1,
        "PropertySearchTypeId":1,
        "TransactionTypeID":2,
        "BuildingTypeId":0,
        "LongitudeMin":f'{lngmin}',
        "LongitudeMax":f'{lngmax}',
        "LatitudeMin":f'{latmin}',
        "LatitudeMax":f'{latmax}',
        "AirCondition":1,
        "PriceMin":250000,
        "RecordsPerPage": records_per_page,
        "CurrentPage": page + 1
    }

    response = requests.post(url, data=body).json() 
    
    for index, mls_post in enumerate(response['Results']):
        if page > 0:
            index += (MAX_RECORDS * page)
            
        try:                        
            lng = float(mls_post['Property']['Address']['Longitude'])
            address = mls_post['Property']['Address']
    
            if (lng >= lngmin and lng <= lngmax and address['AddressText'] not in maindf['Address'].unique()):
                maindf.loc[index,"Address"] = address['AddressText']        
                maindf.loc[index,"Mls Number"]= mls_post['MlsNumber']
                maindf.loc[index,"Date Updated"] = mls_post['Individual'][0]['Organization']['PhotoLastupdate']
                    
                try:
                    maindf.loc[index,"Sale Price"] = mls_post['Property']['Price']
                    maindf.loc[index,"Near By Ammenities"] = mls_post['Property']['AmmenitiesNearBy']
                    maindf.loc[index,"Ownership Type"] = mls_post['Property']['OwnershipType']
                    maindf.loc[index,"Parking"] = mls_post['Property']['Parking'][0]['Name']
                    maindf.loc[index,"Bathrooms"] = mls_post['Building']['BathroomTotal']
                    maindf.loc[index,"Bedrooms"] = mls_post['Building']['Bedrooms']
                except:
                    maindf.loc[index,"Sale Price"] = ""   
                    maindf.loc[index,"Near By Ammenities"] = ""
                    maindf.loc[index,"Ownership Type"] = ""
                    maindf.loc[index,"Parking"] = ""
                    maindf.loc[index,"Bathrooms"] = ""
                    maindf.loc[index,"Bedrooms"] = ""
                
        except Exception as e:
            print('Error ', e)  
            
maindfclean = maindf.loc[maindf['Address']!="",:]

ConnectionError: HTTPSConnectionPool(host='api2.realtor.ca', port=443): Max retries exceeded with url: /Listing.svc/PropertySearch_Post (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000025B5C679CC0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [61]:
maindf.dropna(how='any')

,Address,Sale Price,Mls Number,Near By Ammenities,Ownership Type,Parking,Bathrooms,Bedrooms,Date Updated
0,"#628 -2737 KEELE ST|Toronto, Ontario M3M2E9","$283,900",W4652611,"Hospital, Park, Public Transit, Schools",Condominium/Strata,Underground,1,1,2019-12-15 2:24:42 PM
1,"#614 -2737 KEELE ST|Toronto, Ontario M3M2E9","$294,500",W4653141,"Hospital, Public Transit",Condominium/Strata,Underground,1,1,2019-12-13 6:36:20 PM
2,"#218 -2737 KEELE ST|Toronto, Ontario M3M2E9",,W4653641,,,,,,2019-12-15 6:18:07 PM
3,"#523 -2737 KEELE ST W|Toronto, Ontario M3M2E9",,W4645203,,,,,,2019-11-30 8:24:08 AM
4,"#910 -2737 KEELE ST|Toronto, Ontario M3M2E9","$339,700",W4641655,"Hospital, Park, Public Transit, Schools",Condominium/Strata,Underground,1,2,2019-12-05 2:31:41 PM
5,"#907 -234 ALBION RD|Toronto, Ontario M9W6A5","$355,000",W4622564,"Hospital, Park, Schools",Condominium/Strata,Underground,1,2,2019-12-09 8:24:39 AM
6,"601 - 3555 Bathurst Street|Toronto, Ontario M...",,30778878,,,,,,2019-11-20 9:39:39 AM
7,"#601 -3555 BATHURST ST|Toronto, Ontario M6A2Y8","$365,000",C4637991,"Park, Public Transit, Schools",Condominium/Strata,Underground,2,2,2019-12-06 2:25:51 PM
8,"#201 -2154 DUNDAS ST W|Toronto, Ontario M6R1X3",,W4652489,,,,,,2019-12-13 6:36:03 PM
9,"#111 -234 ALBION RD|Toronto, Ontario M9W6A5","$379,999",W4618026,"Hospital, Public Transit, Schools",Condominium/Strata,Underground,2,3,2019-12-09 2:08:48 PM


In [53]:
maindfclean = maindf.loc[maindf['Address']!="",:]
maindfclean.set_index('Mls Number')
maindfclean.head()

,Address,Sale Price,Mls Number,Near By Ammenities,Ownership Type,Parking,Bathrooms,Bedrooms,Date Updated
0,"#628 -2737 KEELE ST|Toronto, Ontario M3M2E9","$283,900",W4652611,"Hospital, Park, Public Transit, Schools",Condominium/Strata,Underground,1,1,2019-12-15 2:24:42 PM
1,"#614 -2737 KEELE ST|Toronto, Ontario M3M2E9","$294,500",W4653141,"Hospital, Public Transit",Condominium/Strata,Underground,1,1,2019-12-13 6:36:20 PM
2,"#218 -2737 KEELE ST|Toronto, Ontario M3M2E9",,W4653641,,,,,,2019-12-15 6:18:07 PM
3,"#523 -2737 KEELE ST W|Toronto, Ontario M3M2E9",,W4645203,,,,,,2019-11-30 8:24:08 AM
4,"#910 -2737 KEELE ST|Toronto, Ontario M3M2E9","$339,700",W4641655,"Hospital, Park, Public Transit, Schools",Condominium/Strata,Underground,1,2,2019-12-05 2:31:41 PM


In [9]:
pattern = re.compile(r'(\d\d\d\d-\d\d-\d\d)')

In [10]:
maindfclean['Date Updated'] = maindfclean['Date Updated'].str.extract(pattern)


C:\Users\HP Book Pro\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
maindfclean.head()

,Address,Sale Price,Mls Number,Date Updated
0,"#1112 -50 LOTHERTON PTWY|Toronto, Ontario M6B2G7","$275,000",W4648239,2019-12-07
1,"#628 -2737 KEELE ST|Toronto, Ontario M3M2E9","$283,900",W4652611,2019-12-15
2,"#614 -2737 KEELE ST|Toronto, Ontario M3M2E9","$294,500",W4653141,2019-12-13
3,"#218 -2737 KEELE ST|Toronto, Ontario M3M2E9","$294,500",W4653641,2019-12-15
4,"#1018 -1 MASSEY SQ|Toronto, Ontario M4C5L4","$299,999",E4653825,2019-12-16


In [12]:
maindfclean['Sale Price'] = maindfclean['Sale Price'].str[1:]
maindfclean.head()

C:\Users\HP Book Pro\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Address,Sale Price,Mls Number,Date Updated
0,"#1112 -50 LOTHERTON PTWY|Toronto, Ontario M6B2G7","275,000",W4648239,2019-12-07
1,"#628 -2737 KEELE ST|Toronto, Ontario M3M2E9","283,900",W4652611,2019-12-15
2,"#614 -2737 KEELE ST|Toronto, Ontario M3M2E9","294,500",W4653141,2019-12-13
3,"#218 -2737 KEELE ST|Toronto, Ontario M3M2E9","294,500",W4653641,2019-12-15
4,"#1018 -1 MASSEY SQ|Toronto, Ontario M4C5L4","299,999",E4653825,2019-12-16
